In [14]:
import psycopg2 as psy
from datetime import datetime, timedelta

In [4]:
import itertools as it
import psycopg2.sql as sql
from osgeo import ogr
from osgeo import osr
import json

source = osr.SpatialReference()
source.ImportFromEPSG(4326)

target = osr.SpatialReference()
target.ImportFromEPSG(3003)

transform = osr.CoordinateTransformation(source, target)

def map_to_monte_mario(wkt):
    geom = ogr.CreateGeometryFromWkt(wkt)
    geom.Transform(transform)
    return geom.ExportToWkt()

In [2]:
con = psy.connect("dbname=testdb host=timescaledb user=postgres password=foobar")

In [ ]:
SQL = """
SELECT id, stype, ST_AsText(ST_Transform(geom,4326)) AS loc, description from sensors
WHERE ST_DWithin(geom, ST_GeomFromText(%s, 3003), %s) LIMIT 10;
"""

with con:
    with con.cursor() as cur:
        cur.execute(SQL, (map_to_monte_mario('POINT(9.121661 39.223841)'), 10000))
        for r in cur.fetchall():
            print(r)

In [9]:
SQL = """
SELECT tag, description from sensor_types;
"""

with con:
    with con.cursor() as cur:
        cur.execute(SQL)
        for r in cur.fetchall():
            print(r)

('cbca3889-2731-4d2c-b3e9-4b6f1c233bfe', {'key': 'value0'})
('ad31e8de-b5de-4b82-b62b-5ef89a346e55', {'key': 'value1'})
('d00ffa19-fe36-4e8a-949f-5b58b2fae3db', {'key': 'value2'})
('9f92f84d-e4bf-4865-a932-601494245380', {'key': 'value3'})
('1f5505d9-0bb9-45d3-ad72-5dec7d936aaa', {'key': 'value4'})
('ad414722-4ac5-4585-97c2-df92da9a104a', {'key': 'value5'})
('dcf8bc53-4087-4c56-8426-8792c35aa446', {'key': 'value6'})
('1e4f8925-1fa8-4c2a-8d46-b1a4a75dee14', {'key': 'value7'})
('8f3bc219-3604-4d30-bf50-642ded6638e6', {'key': 'value8'})
('dbdeb9e9-e32b-4a79-aa4b-d1fada473dec', {'key': 'value9'})
('f4f6f641-a45c-4db8-a0fe-5a0591812daf', {'key': 'value10'})
('7d2dd228-f2ff-4135-8ecb-6667e28263fb', {'key': 'value11'})
('b8db25e8-2ded-4531-97e6-14cead1fcc0b', {'key': 'value12'})
('d11157df-342f-4e8b-a58e-d1dcad7737b4', {'key': 'value13'})
('ae466b0f-cc12-4b02-a9ad-9e03caceb6f8', {'key': 'value14'})
('362a10d3-0cbd-4a71-b729-54485409907f', {'key': 'value15'})
('1b7c6a24-a7c8-47c5-b3a3-df70c07b

In [44]:
def get_sensors_in_timespace(con, center, radius, start, stop):
    SQL = """
    WITH spatial AS (
         SELECT id, stype, ST_AsText(ST_Transform(geom,4326)) AS loc, description
         FROM sensors 
         WHERE ST_DWithin(geom, ST_GeomFromText(%s, 3003), %s)
        ), selected AS (
           SELECT DISTINCT ON (sensor) sensor FROM measures m
           WHERE m.time > %s AND m.time < %s
                 AND m.sensor IN (SELECT id FROM spatial)
        )
    SELECT spatial.id, spatial.loc, spatial.description, sensor_types.tag, sensor_types.description 
    FROM spatial, sensor_types
    WHERE  spatial.id in (SELECT sensor FROM selected) AND sensor_types.id = spatial.stype;
    """

    with con:
        with con.cursor() as cur:
            cur.execute(SQL, (map_to_monte_mario(center), radius, start, stop))
            return [r for r in cur.fetchall()]

In [54]:
cagliari = 'POINT(9.121661 39.223841)'
radius = 10000 # meters
start = '2019-02-21'
stop = '2019-02-22'

In [55]:
%timeit sensors = get_sensors_in_timespace(con, cagliari, radius, start, stop)

344 ms ± 4.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
def get_timeseries(con, sensor_id, start=None, stop=None, bucket=None, op=None, isoformat=False):
    SQL = """
    SELECT time, value FROM measures m
    WHERE m.sensor = %s;
    """

    with con:
        with con.cursor() as cur:
            cur.execute(SQL, (5902,))
            if isoformat:
                return [[r[0].isoformat(), r[1]] for r in cur.fetchall()]

In [42]:
%timeit s5904 = get_timeseries(5904, isoformat=True)

106 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
